## 0.0. setup

1. follow the setup instructions in `1_0_hybridization_setup.ipynb`

2. create directories

In [1]:
!mkdir -p ~/hybridization_data/databases_hybrid
!rm -rf ~/hybridization_data/databases_pickle 
!mkdir -p ~/hybridization_data/databases_pickle
!cp -a /srv/data/autumn_school/hybridization/databases_pickle/* ~/hybridization_data/databases_pickle

## 0.1. imports
### 0.1.1. regular imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import brightway2 as bw
import ecospold2matrix as e2m
import pymrio
# data science
import pandas as pd
# deep copy
import copy

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/weinold/bw_data


### 0.1.2. local imports

In [3]:
sys.path.append(os.path.join(Path.home(), 'pylcaio', 'src')) # required for local import of pylcaio
import pylcaio

## 0.2. file paths
### 0.2.1. directories

In [4]:
%%capture
print(path_dir_databases_raw := os.path.join(Path.home(), 'hybridization_data/databases_raw'))
print(path_dir_databases_pickle := os.path.join(Path.home(), 'hybridization_data/databases_pickle'))
print(path_dir_databases_hybrid := os.path.join(Path.home(), 'hybridization_data/databases_hybrid'))

### 0.2.2. databases

In [5]:
%%capture
# Exiobase
print(path_file_exiobase := os.path.join(path_dir_databases_pickle, 'exiobase_monetary_pxp_2012.pickle'))
print(path_file_exiobase_capital_use_matrix := os.path.join(path_dir_databases_raw, 'bar_exio_v3_6_2011pxp.mat')) # from https://zenodo.org/record/3874309
# Ecoinvent
print(path_file_ecoinvent := os.path.join(path_dir_databases_pickle, 'ecoinvent_3_5_cutoffPandas_symmNorm.gz.pickle'))
# Hybrid
print(name_file_hybrid := 'hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.pickle')
print(path_file_hybrid := os.path.join(path_dir_databases_hybrid, name_file_hybrid))

## 0.3. load `pickle` files

In [6]:
with gzip.open(path_file_ecoinvent, 'rb') as filestream:
    ecoinvent: dict = pd.read_pickle(filestream)

In [7]:
with open(path_file_exiobase, 'rb') as filestream:
    exiobase: pymrio.core.mriosystem.IOSystem = pd.read_pickle(filestream)

## 1.1. `pylcaio` hybridization

### 1.1.1. hybridization

In [8]:
database_loader: pylcaio.DatabaseLoader  = pylcaio.DatabaseLoader(
    lca_database_processed = ecoinvent,
    io_database_processed = exiobase,
    lca_database_name_and_version = 'ecoinvent3.5',
    io_database_name_and_version = 'exiobase3'
)

In [9]:
lcaio_object: pylcaio.LCAIO = database_loader.combine_ecoinvent_exiobase(
    path_to_capitals = path_file_exiobase_capital_use_matrix,
    complete_extensions = True,
    impact_world = True,
    regionalized = False
)

In [ ]:
lcaio_object = copy.deepcopy(lcaio_object) # to avoid errors when re-running cells
lcaio_object.hybridize(
    price_neutral_cut_off_matrix = 'STAM',
    capitals = False,
    priceless_scaling = True
)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...


/home/weinold/pylcaio/src/pylcaio.py:874: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.H = self.H.append([self.H] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))


Building geography concordance...
Filter H matrix...
Build Cut-off matrix...
Add processes with 'priceless scaling' to Cut-off matrix...


### 1.1.2. save files

In [ ]:
lcaio_object.save_system( # this method must be used, otherwise pickle files cannot be read correcly by pylcaio.Analysis
    file_name = name_file_hybrid,
    file_path = path_dir_databases_hybrid,
    format = 'pickle'
)

Database saved to /home/weinold/hybridization_data/databases_hybrid/hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.pickle
Description file saved to /home/weinold/hybridization_data/databases_hybrid/description_hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.txt
